In [13]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd
import matplotlib.pyplot as plt 

urls=["https://www.cricviz.com/ipl-powerplay-kings-bowling/"]
headers_={'user agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

for url in urls:
  page=requests.get(url, headers=headers_)
  
  soup=bs(page.content,'html.parser')

  Contents=soup.findAll("p")

Scrapped_data=[]
for content in Contents:
  Text=content.get_text()
  Scrapped_data.append([Text])

df=pd.DataFrame(Scrapped_data,columns=["Text"])

display(df)

df.to_json("ipl bowler.json")

,Text
0,CricViz
1,CricViz analyst Rufus Bullough analyses the mo...
2,The dotted lines in the scatter graph below re...
3,Mumbai Indians and Sunrisers Hyderabad have be...
4,Deepak Chahar has been incredibly influential ...
5,No pace bowler has taken wickets more often th...
6,Bowling spin in the Powerplay is considered mo...
7,Nobody has taken more Powerplay wickets bowlin...
8,Arguably the biggest impact in Powerplay spin ...
9,Rufus Bullough is an analyst at CricViz


In [14]:
import string
string.punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df['puntuation free text']= df['Text'].apply(lambda x:remove_punctuation(x))
#Lowering the text
df['lower']= df['puntuation free text'].apply(lambda x: x.lower())

#Tokenization
from nltk.tokenize import TweetTokenizer as tt
#applying function to the column
tokenizer = tt()      
df['tokenized_text'] = df['lower'].apply(lambda x: tokenizer.tokenize(x))

#Removing stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopword = stopwords.words('english')
def remove_stopwords(text):
    output= [i for i in text if i not in stopword]
    return output
df['no_stopwords']= df['tokenized_text'].apply(lambda x:remove_stopwords(x))

#Lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(no_stopwords):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in no_stopwords]
    return lemm_text
df['lematizer']=df['no_stopwords'].apply(lambda x:lemmatizer(x))
display(df.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Text,puntuation free text,lower,tokenized_text,no_stopwords,lematizer
0,CricViz,CricViz,cricviz,[cricviz],[cricviz],[cricviz]
1,CricViz analyst Rufus Bullough analyses the mo...,CricViz analyst Rufus Bullough analyses the mo...,cricviz analyst rufus bullough analyses the mo...,"[cricviz, analyst, rufus, bullough, analyses, ...","[cricviz, analyst, rufus, bullough, analyses, ...","[cricviz, analyst, rufus, bullough, analysis, ..."
2,The dotted lines in the scatter graph below re...,The dotted lines in the scatter graph below re...,the dotted lines in the scatter graph below re...,"[the, dotted, lines, in, the, scatter, graph, ...","[dotted, lines, scatter, graph, represent, lea...","[dotted, line, scatter, graph, represent, leag..."
3,Mumbai Indians and Sunrisers Hyderabad have be...,Mumbai Indians and Sunrisers Hyderabad have be...,mumbai indians and sunrisers hyderabad have be...,"[mumbai, indians, and, sunrisers, hyderabad, h...","[mumbai, indians, sunrisers, hyderabad, econom...","[mumbai, indian, sunrisers, hyderabad, economi..."
4,Deepak Chahar has been incredibly influential ...,Deepak Chahar has been incredibly influential ...,deepak chahar has been incredibly influential ...,"[deepak, chahar, has, been, incredibly, influe...","[deepak, chahar, incredibly, influential, csk,...","[deepak, chahar, incredibly, influential, csk,..."


In [15]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
import nltk
from nltk.corpus import stopwords

#Vis
import pyLDAvis
import pyLDAvis.gensim

import warnings 
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [17]:
def load_data(file):
  with open(file,"r",encoding="utf-8") as f:
    data=json.load(f)
  return(data)

def write_data(file,data):
  with open(file,"w",encoding="utf-8") as f:
    json.dump(data,f,indent=4)

In [18]:
nltk.download("stopwords")
print(stopwords)

<WordListCorpusReader in '/root/nltk_data/corpora/stopwords'>


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
data=load_data("/content/ipl bowler.json")["Text"]
data=data["10"]
data=data.split(".")
data

['If you have any questions or just want to say hi, please feel free to send us an email',
 '']

In [20]:
def lemmatization(texts,allowed_postages=["NOUN","ADJ","VERB","ADV"]):
  nlp=spacy.load("en_core_web_sm",disable=["senter","ner","parser"])
  texts_out=[]
  for text in texts:
    doc=nlp(text)
    new_text=[]
    for token in doc:
      if token.pos_ in allowed_postages:
        new_text.append(token.lemma_)
    final=" ".join(new_text)
    texts_out.append(final)
  return (texts_out)

lemmatized_texts=lemmatization(data)
lemmatized_texts

['question just want say feel free send email', '']

In [21]:
def gen_words(texts):
  final=[]
  for text in texts:
    new=gensim.utils.simple_preprocess(text,deacc=True)
    final.append(new)
  return(final)

data_words=gen_words(lemmatized_texts)
print(data_words)

[['question', 'just', 'want', 'say', 'feel', 'free', 'send', 'email'], []]


In [22]:
id2word=corpora.Dictionary(data_words)
corpus=[]
for text in data_words:
  new = id2word.doc2bow(text)
  corpus.append(new)

print(corpus)

word = id2word
print(word)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], []]
Dictionary<8 unique tokens: ['email', 'feel', 'free', 'just', 'question']...>


In [23]:
lda_model= gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=30,random_state=100,update_every=1,
                                           chunksize=100,passes=10,alpha="auto")

In [24]:
pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim.prepare(lda_model,corpus,id2word,mds="mmds",R=30)
vis

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.9/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/manifold/_mds.py:166: RuntimeWarning: divide by zero encountered in double_scalars
  old_stress = stress / dis
/usr/local/lib/python3.9/dist-packages/sklearn/manifold/_mds.py:162: RuntimeWarning: invalid value encountered in double_scalars
  if (old_stress - stress / dis) < eps:
/usr/local/lib/python3.9/dist-packages/sklearn/manifold/_mds.py:166: RuntimeWarning: invalid value encountered in double_scalars
  old_stress = stress / dis
/usr/local/

PreparedData(topic_coordinates=         x    y  topics  cluster       Freq
topic                                      
18     0.0  0.0       1        1  91.191363
0      0.0  0.0       2        1   0.303746
1      0.0  0.0       3        1   0.303746
28     0.0  0.0       4        1   0.303746
27     0.0  0.0       5        1   0.303746
26     0.0  0.0       6        1   0.303746
25     0.0  0.0       7        1   0.303746
24     0.0  0.0       8        1   0.303746
23     0.0  0.0       9        1   0.303746
22     0.0  0.0      10        1   0.303746
21     0.0  0.0      11        1   0.303746
20     0.0  0.0      12        1   0.303746
19     0.0  0.0      13        1   0.303746
17     0.0  0.0      14        1   0.303746
16     0.0  0.0      15        1   0.303746
15     0.0  0.0      16        1   0.303746
14     0.0  0.0      17        1   0.303746
13     0.0  0.0      18        1   0.303746
12     0.0  0.0      19        1   0.303746
11     0.0  0.0      20        1   0.303746
10     0.0  0.0      21        1   0.303746
9      0.0  0.0      22        1   0.303746
8      0.0  0.0      23        1   0.303746
7      0.0  0.0      24        1   0.303746
6      0.0  0.0      25        1   0.303746
5      0.0  0.0      26        1   0.303746
4      0.0  0.0      27        1   0.303746
3      0.0  0.0      28        1   0.303746
2      0.0  0.0      29        1   0.303746
29     0.0  0.0      30        1   0.303746, topic_info=        Term      Freq  Total Category  logprob  loglift
0      email  1.000000    1.0  Default   8.0000      8.0
1       feel  1.000000    1.0  Default   7.0000      7.0
2       free  1.000000    1.0  Default   6.0000      6.0
3       just  1.000000    1.0  Default   5.0000      5.0
4   question  1.000000    1.0  Default   4.0000      4.0
..       ...       ...    ...      ...      ...      ...
3       just  0.003037    1.0  Topic30  -2.0794      0.0
4   question  0.003037    1.0  Topic30  -2.0794      0.0
5        say  0.003037    1.0  Topic30  -2.0794      0.0
6       send  0.003037    1.0  Topic30  -2.0794      0.0
7       want  0.003037    1.0  Topic30  -2.0794      0.0

[248 rows x 6 columns], token_table=      Topic  Freq      Term
term                       
0         1   1.0     email
1         1   1.0      feel
2         1   1.0      free
3         1   1.0      just
4         1   1.0  question
5         1   1.0       say
6         1   1.0      send
7         1   1.0      want, R=8, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 1, 2, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 30])